[Source Code](https://github.com/random-forests/tutorials/blob/master/decision_tree.ipynb)

def build_tree(rows): <br/>
       
       info, question = find_best_split(rows)
       
       if info == 0: return Leaf(rows)
       
       true_rows, false_rows = 
           patrition(rows, question)
       
       true_branch = build_tree(true_rows)
       
       false_branch = build_tree(false_rows)
       
       return Decision_Node(question, true_branch, false_branch)
              

In [1]:
# data
training_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon']
]

In [2]:
# Column Labels
header = ["color", "diameter", "label"]

In [3]:
def unique_vals(rows, col):
    """Find the unique values for a column in a dataset"""
    return set([row[col] for row in rows])

In [6]:
unique_vals(training_data, 1)

{1, 3}

In [33]:
def class_counts(rows):
    """Counts the number of each type of example in dataset."""
    counts = {} # a dictionary of label -> count.
    for row in rows:
        # in our dataset format, the label is alway the last column
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [8]:
class_counts(training_data)

{'Apple': 2, 'Grape': 2, 'Lemon': 1}

In [9]:
def is_numeric(value):
    """Test if a value is numeric."""
    return isinstance(value, int) or isinstance(value, float)

In [10]:
is_numeric(10.0)

True

In [12]:
class Question:
    """A Question is used to partition a dataset.
    
    This class just records a 'column number ' (e.g., 0 for Color) and a
    'column value' (e.g., Green). The 'match' method is used to compare
    the feature value in an example to the feature value stored in the 
    question.
    """
    def __init__(self, column, value):
        self.column = column
        self.value = value
        
    def match(self, example):
        # Compare the feature value in an example to the
        # feature value in this question
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value
        
    def __repr__(self):
        # This is a helper method to print
        # the question in a readable format.
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))

In [14]:
Question(0, "Green")

Is color == Green?

In [15]:
q = Question(0, "Green")
q

Is color == Green?

In [20]:
example = training_data[0]
print(example[0])
q.match(example)

Green


True

In [23]:
def partition(rows, question):
    """Partitions a dataset.
    
    For each row in the dataset, check if it matches the question. If
    so, add it to 'true rows', otherwise, add it to ' false rows'.
    """
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [24]:
true_rows, false_rows = partition(training_data, Question(0, 'Red'))

In [25]:
true_rows

[['Red', 1, 'Grape'], ['Red', 1, 'Grape']]

In [26]:
false_rows

[['Green', 3, 'Apple'], ['Yellow', 3, 'Apple'], ['Yellow', 3, 'Lemon']]

In [37]:
def gini(rows):
    """Calculate the Gini Impurity for a list of rows.
    
    There are a few different ways to do this, I thought this one was
    the most concise.
    """
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity  

In [38]:
no_mixing = [['Apple'],
            ['Apple']]
gini(no_mixing)

0.0

In [39]:
some_mixing = [['Apple'],
              ["Orange"]]
gini(some_mixing)

0.5

In [40]:
def info_gain(left, right, current_uncertainty):
    """Information Gain.
    
    The uncertainty of the starting node, minus the weighted impurity of
    two child nodes.
    """
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

In [41]:
current_uncertainty = gini(training_data)
current_uncertainty

0.6399999999999999

In [64]:
def find_best_split(rows):
    """Find the best question to ask by iterating over every feature / value
    and calculating the information gain."""
    best_gain = 0
    best_question = None
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1 # number of columns
    
    for col in range(n_features): # for each feature
        
        values = set([row[col] for row in rows]) # unique values in the column
        
        for val in values: # for each value
            
            question = Question(col, val)
            
            # try splitting the dataset
            true_rows, false_rows = partition(rows, question)
            
            # Skip this split if it doesn't divide the dataset
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue
                
            # Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)
            
            # Actually, we can use '>' instead of '>=' here
            if gain > best_gain:
                best_gain, best_question = gain, question
                
    return best_gain, best_question

In [65]:
find_best_split(training_data)

['Green', 3, 'Apple']


(0.37333333333333324, Is color == Red?)

In [46]:
class Leaf:
    """A Leaf node classifies data.
    
    This holds a dictionary of class (e.g., "Apple") -> number of times
    it appears in the rows from the training data that reach this leaf.
    """
    
    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [47]:
class Decision_Node:
    """A Decision Node asks a question.
    
    This holds a reference to the question, and to the two child nodes.
    """
    
    def __init__(self,
                question,
                true_branch,
                false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [50]:
def build_tree(rows):
    """Builds the tree.
    
    Rules of recursion: 1) Believe that it works. 2) Start by checking
    for the base case (no further information gain). 3) Prepare for
    giant stack traces.
    """
    
    # Try partitioning the dataset on each of the unique attribute,
    # calculate the information gain,
    # and return the question that produces the highest gain.
    gain, question = find_best_split(rows)
    
    # Base Case: no further info gain
    # Since we can ask no further questions,
    # we'll return a leaf.
    if gain == 0:
        return Leaf(rows)
    
    # If we reach here, we have found a useful feature / value
    # to partition on.
    true_rows, false_rows = partition(rows, question)
    
    # Recursively build the true branch.
    true_branch = build_tree(true_rows)
    
    # Recursively build the false branch
    false_branch = build_tree(false_rows)
    
    # Return a Question node.
    # This records the best feature / value to ask at this point,
    # as well as the branches to follow
    # depending on the answer.
    return Decision_Node(question, true_branch, false_branch)

In [49]:
def print_tree(node, spacing=""):
    """Tree Printing function."""
    
    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print(spacing + "Predict", node.predictions)
        return
    
    # Print the question at this node
    print (spacing + str(node.question))
    
    # Call this function recursively on the true branch
    print(spacing + "--> True:")
    print_tree(node.true_branch, spacing + "  ")
    
    # Call this function recursively on the false branch
    print(spacing + "--> False:")
    print_tree(node.false_branch, spacing + "  ")

In [51]:
my_tree = build_tree(training_data)

In [52]:
type(my_tree)

__main__.Decision_Node

In [53]:
print_tree(my_tree)

Is color == Red?
--> True:
  Predict {'Grape': 2}
--> False:
  Is color == Yellow?
  --> True:
    Predict {'Apple': 1, 'Lemon': 1}
  --> False:
    Predict {'Apple': 1}


In [54]:
def classify(row, node):
    """See the 'rules of recursion' above."""
    
    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        return node.predictions
    
    # Decide whether to follow the true-branch or the false-branch.
    # Compare the feature / value stored in the node,
    # to the example we're considering.
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [56]:
classify(training_data[1], my_tree)

{'Apple': 1, 'Lemon': 1}

In [57]:
def print_leaf(counts):
    """A nicer way to print the predictions at a leaf."""
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [59]:
print_leaf(classify(training_data[1], my_tree))

{'Apple': '50%', 'Lemon': '50%'}

In [60]:
# Evaluate
testing_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 4, 'Apple'],
    ['Red', 2, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon']
]

In [61]:
for row in testing_data:
    print("Actual: %s. Predicted: %s" %
         (row[-1], print_leaf(classify(row, my_tree))))

Actual: Apple. Predicted: {'Apple': '100%'}
Actual: Apple. Predicted: {'Apple': '50%', 'Lemon': '50%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Lemon. Predicted: {'Apple': '50%', 'Lemon': '50%'}
